# Recap & creating the empty weather table in the database

In [1]:
# Get weather data

import pandas as pd
import requests
from pytz import timezone
from datetime import datetime
from keys import sql
from key import api
def retrieve_data():
    connection_string = create_connection_string()
    cities_df = fetch_cities_data(connection_string)
    weather_df = fetch_weather_data(cities_df)
    return weather_df

def create_connection_string():
    schema = "gans_local" # The name of your database
    host = "127.0.0.1"
    user = "root"
    password = sql # Your MySQL password
    port = 3306
    return f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

def fetch_cities_data(connection_string):
    return pd.read_sql("cities", con=connection_string)

def fetch_weather_data(cities_df):

    berlin_timezone = timezone('Europe/Berlin')
    API_key = api #USE YOUR OWN OPEN WHEATHER KEY HERE
    weather_items = []

    for _, city in cities_df.iterrows():

        latitude = city["Latitude"]
        longitude = city["Longitude"]
        city_id = city["City_id"]

        url = (f"https://api.openweathermap.org/data/2.5/forecast?lat={latitude}&lon={longitude}&appid={API_key}&units=metric")
        response = requests.get(url)
        weather_data = response.json()

        retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")

        for item in weather_data["list"]:
            weather_item = {
              "city_id": city_id,
              "forecast_time": item.get("dt_txt"),
              "outlook": item["weather"][0].get("description", None),
              "temperature": item["main"].get("temp"),
              "rain_in_last_3h": item.get("rain", {}).get("3h", 0),
              "wind_speed": item["wind"].get("speed"),
              "rain_prob": item.get("pop", None),
              "data_retrieved_at": retrieval_time
            }
            weather_items.append(weather_item)

    weather_df = pd.DataFrame(weather_items)
    weather_df["forecast_time"] = pd.to_datetime(weather_df["forecast_time"])
    weather_df["data_retrieved_at"] = pd.to_datetime(weather_df["data_retrieved_at"])

    return weather_df

In [3]:
retrieve_data()

,city_id,forecast_time,outlook,temperature,rain_in_last_3h,wind_speed,rain_prob,data_retrieved_at
0,1,2024-10-07 18:00:00,few clouds,14.68,0.0,1.88,0.0,2024-10-07 18:46:26
1,1,2024-10-07 21:00:00,scattered clouds,14.40,0.0,1.95,0.0,2024-10-07 18:46:26
2,1,2024-10-08 00:00:00,broken clouds,14.03,0.0,2.41,0.0,2024-10-07 18:46:26
3,1,2024-10-08 03:00:00,overcast clouds,13.89,0.0,2.67,0.0,2024-10-07 18:46:26
4,1,2024-10-08 06:00:00,overcast clouds,13.63,0.0,2.88,0.0,2024-10-07 18:46:26
...,...,...,...,...,...,...,...,...
115,3,2024-10-12 03:00:00,broken clouds,4.57,0.0,1.32,0.0,2024-10-07 18:46:27
116,3,2024-10-12 06:00:00,broken clouds,5.16,0.0,1.32,0.0,2024-10-07 18:46:27
117,3,2024-10-12 09:00:00,overcast clouds,7.19,0.0,1.39,0.0,2024-10-07 18:46:27
118,3,2024-10-12 12:00:00,broken clouds,10.65,0.0,0.76,0.0,2024-10-07 18:46:27


In [ ]:
retrieve_data().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   city_id            120 non-null    int64         
 1   forecast_time      120 non-null    datetime64[ns]
 2   outlook            120 non-null    object        
 3   temperature        120 non-null    float64       
 4   rain_in_last_3h    120 non-null    float64       
 5   wind_speed         120 non-null    float64       
 6   rain_prob          120 non-null    float64       
 7   data_retrieved_at  120 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(4), int64(1), object(1)
memory usage: 7.6+ KB


```sql
CREATE TABLE weather (
	weather_id INT AUTO_INCREMENT,
    city_id INT NOT NULL,
    forecast_time DATETIME,
    outlook VARCHAR(255),
    temperature FLOAT,
    rain_in_last_3h FLOAT,
    wind_speed FLOAT,
    rain_prob FLOAT,
    data_retrieved_at DATETIME,
    PRIMARY KEY (weather_id),
    FOREIGN KEY (city_id) REFERENCES cities(City_id)
);
```

# Making our function to push the weather_df to the SQL weather table

In [5]:
# Get weather data and push it to the eather table in the database

import pandas as pd
import requests
from pytz import timezone
from datetime import datetime
from keys import sql
from key import api

def retrieve_data():
    connection_string = create_connection_string()
    cities_df = fetch_cities_data(connection_string)
    weather_df = fetch_weather_data(cities_df)
    push_weather_data_to_database(weather_df, connection_string)
    return "Data has been updated"

def create_connection_string():
    schema = "gans_local" # The name of your database
    host = "127.0.0.1"
    user = "root"
    password = sql # Your MySQL password
    port = 3306
    return f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

def fetch_cities_data(connection_string):
    return pd.read_sql("cities", con=connection_string)

def fetch_weather_data(cities_df):

    berlin_timezone = timezone('Europe/Berlin')
    API_key = api #USE YOUR OWN OPEN WHEATHER KEY HERE
    weather_items = []

    for _, city in cities_df.iterrows():

        latitude = city["Latitude"]
        longitude = city["Longitude"]
        city_id = city["City_id"]

        url = (f"https://api.openweathermap.org/data/2.5/forecast?lat={latitude}&lon={longitude}&appid={API_key}&units=metric")
        response = requests.get(url)
        weather_data = response.json()

        retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")

        for item in weather_data["list"]:
            weather_item = {
              "city_id": city_id,
              "forecast_time": item.get("dt_txt"),
              "outlook": item["weather"][0].get("description", None),
              "temperature": item["main"].get("temp"),
              "rain_in_last_3h": item.get("rain", {}).get("3h", 0),
              "wind_speed": item["wind"].get("speed"),
              "rain_prob": item.get("pop", None),
              "data_retrieved_at": retrieval_time
            }
            weather_items.append(weather_item)

    weather_df = pd.DataFrame(weather_items)
    weather_df["forecast_time"] = pd.to_datetime(weather_df["forecast_time"])
    weather_df["data_retrieved_at"] = pd.to_datetime(weather_df["data_retrieved_at"])

    return weather_df

def push_weather_data_to_database(weather_df, connection_string):

    weather_df.to_sql('weather',
                    if_exists='append',
                    con=connection_string,
                    index=False)

In [7]:
retrieve_data()

'Data has been updated'

## Reading the weather table from the Database

In [9]:
schema = "gans_local" # The name of your database
host = "127.0.0.1"
user = "root"
password = sql # Your MySQL password
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [11]:
weather_df = pd.read_sql("weather", con=connection_string)
weather_df

,weather_id,city_id,forecast_time,outlook,temperature,rain_in_last_3h,wind_speed,rain_prob,data_retrieved_at
0,1,1,2024-10-07 18:00:00,few clouds,14.68,0.0,1.88,0.0,2024-10-07 18:48:24
1,2,1,2024-10-07 21:00:00,scattered clouds,14.40,0.0,1.95,0.0,2024-10-07 18:48:24
2,3,1,2024-10-08 00:00:00,broken clouds,14.03,0.0,2.41,0.0,2024-10-07 18:48:24
3,4,1,2024-10-08 03:00:00,overcast clouds,13.89,0.0,2.67,0.0,2024-10-07 18:48:24
4,5,1,2024-10-08 06:00:00,overcast clouds,13.63,0.0,2.88,0.0,2024-10-07 18:48:24
...,...,...,...,...,...,...,...,...,...
115,116,3,2024-10-12 03:00:00,broken clouds,4.57,0.0,1.32,0.0,2024-10-07 18:48:24
116,117,3,2024-10-12 06:00:00,broken clouds,5.16,0.0,1.32,0.0,2024-10-07 18:48:24
117,118,3,2024-10-12 09:00:00,overcast clouds,7.19,0.0,1.39,0.0,2024-10-07 18:48:24
118,119,3,2024-10-12 12:00:00,broken clouds,10.65,0.0,0.76,0.0,2024-10-07 18:48:24
